In [6]:
import cmp
import pdir
from lattices import *
%matplotlib notebook
np.set_printoptions(threshold=np.nan)

In [7]:
# Inputs
eq = np.isclose
# Lattice vectors (3 vectors of length 3)
a = 1
b = 2
a1 = np.array([1, 0, 0])
a2 = np.array([0, 1, 0])
a3 = np.array([0, 0, 1])
theta = 80*np.pi/180

# Array of basis vectors
basis = np.array([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# Colors for each of the basis vectors
blargh = ('r', 'r','b','b')
# Size multiplier for each of the atoms. Default is 1
sizes = (2,2,1,1)
verbose = True


# Gridline type:
# Soft: Lines along cartesian axes. Takes into account nonequal lattice spacing
# LatticeVectors: Lines along the latticevectors (only on lattice points)
GridType = "lattice"

# Limit type:
# individual: Sets the limits as max(nx*a1,ny*a2,nz*a3), so we include nx unitcells in the a1 direction, etc.
# sum: Sets the limits r_min = n_min*[a1 a2 a3] and likewise for n_max
LimType = "dynamic"
Maxs = [2,2,2]
Mins = [0,0,0]

LatticeType = "conventional fcc"

#Lattice(lattice_name = LatticeType, colors = blargh, sizes = sizes, max_ = Maxs, verbose=True)
#Reciprocal(lattice_name=LatticeType, indices=(1,1,0))

In [11]:
def setup_scattering(lattice_name='simple cubic', verbose=False):
    lattice_name = lattice_name.lower()
    min_, max_ = (-2, -2, -1), (2, 2, 1)
    grid_type = latticelines[lattice_name]
    lim_type = "proper"
    colors = ["xkcd:cement",
              "xkcd:cornflower blue",
              "xkcd:cornflower blue",
              "xkcd:cornflower blue"]
    sizes = [1, 1, 1, 1]
    (a1, a2, a3), basis, _ = chooser(lattice_name, verbose=verbose)
    r_min, r_max, n_min, n_max = find_limits(lim_type, a1, a2, a3,
                                                      min_, max_)
    (atomic_positions, lattice_coefficients, atomic_colors, atomic_sizes,
     lattice_position) = generator(a1, a2, a3, basis, colors, sizes,
                                            lim_type, n_min, n_max, r_min,
                                            r_max)
    objects = [atomic_positions, lattice_coefficients, atomic_colors,
               atomic_sizes, lattice_position]
    objects = limiter(atomic_positions, objects, r_min, r_max)
    (atomic_positions, lattice_coefficients, atomic_colors, atomic_sizes,
     lattice_position) = objects
    
    pruned_lines = grid_lines(a1, a2, a3, atomic_positions,
                                       lattice_position, grid_type,
                                       verbose=verbose)
    
    fig = plt.figure()
    ax = fig.gca(projection="3d")

    # Plot atoms. For now a single size and color
    ax.scatter(atomic_positions[:, 0], atomic_positions[:, 1],
               atomic_positions[:, 2], c=atomic_colors, s=atomic_sizes)
    
    g_col = 'k'
    g_w = 0.5
    g_a = 0.6
    
    for line in pruned_lines:
        ax.plot(line[0], line[1], line[2], color=g_col, linewidth=g_w, alpha=g_a)
    
    # Plot the "Beam"
    beam_length = 2
    beam_end_z = 1
    beam = np.array([[0, 0], [0, 0], [beam_length + beam_end_z, beam_end_z]])
    ax.plot(beam[0], beam[1], beam[2], linewidth=2, color='b')
    
    ax.set_aspect('equal')
    ax.set_proj_type('ortho')
    ax.set_xlim([r_min[0], r_max[0]])
    ax.set_ylim([r_min[1], r_max[1]])
    ax.set_zlim([r_min[2], r_max[2]])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.grid(False)
    plt.axis('off')
    plt.axis('equal')


def scattering(a1, a2, a3, basis, scattering_length, k_in):
    basis = np.atleast_2d(basis)
    length_basis = np.shape(basis)
    if len(length_basis) == 1:
        n_basis = 1
    elif len(length_basis) > 1:
        n_basis = length_basis[0]
    
    # First the scaling factor for the reciprocal lattice
    scale = a1.dot(np.cross(a2, a3))
    # Then the reciprocal lattice
    b1 = 2 * np.pi * np.cross(a2, a3) / scale
    b2 = 2 * np.pi * np.cross(a3, a1) / scale
    b3 = 2 * np.pi * np.cross(a1, a2) / scale
    lattice = np.array([b1, b2, b3])
    
    # Then we need to create all the reciprocal lattice vectors
    h_min, j_min, l_min = [-5] * 3
    h_max, j_max, l_max = [5] * 3
    
    # We subtract 1, because we exclude the (0,0,0) index
    num_G = ((h_max + 1 - h_min) * (j_max + 1 - j_min) * (l_max + 1 - l_min)) - 1
    G_array = np.zeros((num_G, 3))
    
    # Create the range of miller indices
    h = range(h_min, h_max + 1)
    j = range(j_min, j_max + 1)
    l = range(l_min, l_max + 1)
    
    # Create the array of indices and exclude the (0,0,0)
    indices = np.asarray(list(itertools.product(h, j ,l)))
    not_zeros = np.sum(indices == 0, 1) != 3
    indices = indices[not_zeros]
    
    # Create the array of reciprocal lattice vectors
    G_array = indices @ lattice
    
    
    # Next we calculate the structure factors (we assume neutron scattering)
    structure_factors = np.zeros(num_G)
    counter = 0
    for G in G_array:
        for n in range(n_basis):
            atom = basis[n,]
            structure_factors[counter] += scattering_length[n] * np.exp(1j * G.dot(atom))
        counter += 1
    intensity = structure_factors * np.conj(structure_factors)
    
    # Then we get the scattered wavevectors:
    k_out = G_array + k_in
    
    # And select only those that actually satisfy |k_in| = |k_out|
    mag_equal = eq(mag(k_out), mag(k_in))
    
    # delete all indices, intensities and k_out, that don't satisfy the above
    indices = indices[mag_equal]
    intensity = intensity[mag_equal]
    k_out = k_out[mag_equal]
    return intensity, k_out, indices


def smart_cubic_k(indices, k0=None, k1=None, k2=None):
    indices = np.array(indices)
    scaler = np.pi * indices.dot(indices)
    
    # Some input sanitization
    if indices.shape != (3,):
        print("We need 3 indices and only 3 indices!")
        return
    
    # More sanitization
    ks = np.array([k0 is None, k1 is None, k2 is None])
    if np.sum(ks) != 1:
        print("You need to specify 2 and only 2 of the k-coordinates!")
        return
    
    # Now that we've made sure only 1 component is unspecified, only one of these if statements will execute
    if ks[0]:
        k0 = - (scaler + indices[1]*k1 + indices[2]*k2)/indices[0]
    if ks[1]:
        k1 = - (scaler + indices[0]*k0 + indices[2]*k2)/indices[1]
    if ks[2]:
        k2 = - (scaler + indices[0]*k0 + indices[1]*k1)/indices[2]
    
    return np.array([k0, k1, k2])


def projection(k_array, r=np.array([0, 0, 0]), n=np.array([0,0,1]), p0=np.array([0,0,5])):
    
    # We assume k_array is a 2D array, with shape (N,3) where N is the number of scattered wave vectors
    num = n.dot(p0-r)
    den = k_array @ n
    d = num / den
    # Next we make sure that there are enough d's: make it 2d:
    D = np.vstack((d, d, d)).T
    p = r + k_array * D
    return p

In [12]:
(a1, a2, a3), basis, _ = chooser(lattice_name='simple cubic')
scattering_length = [1]
k_in = np.array([0,0, -3*np.pi])
intensities, k, indices = scattering(a1, a2, a3, basis, scattering_length, k_in)
points = projection(k)

/home/niko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:105: ComplexWarning: Casting complex values to real discards the imaginary part
